# Getting started

In [1]:
#!pip install deepts_forecasting

### Import libraries

In [1]:
import numpy as np
import pandas as pd
import torch
from deepts_forecasting.utils.data import TimeSeriesDataSet
from torch.utils.data import DataLoader
from deepts_forecasting.models.seq2seq import Seq2SeqNetwork
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from deepts_forecasting.utils.data.encoders import TorchNormalizer
from deepts_forecasting.datasets import AirPassengersDataset


D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset

In [2]:
data = AirPassengersDataset().load()
data['year'] = data['Month'].dt.year
data['month'] = data['Month'].dt.month
data['group'] = '0'
data['time_idx'] = np.arange(len(data))
data['Passengers'] = data['Passengers'].astype(float)
data['month'] = data['month'].astype('str')
data.head()

,Month,Passengers,year,month,group,time_idx
0,1949-01-01,112.0,1949,1,0,0
1,1949-02-01,118.0,1949,2,0,1
2,1949-03-01,132.0,1949,3,0,2
3,1949-04-01,129.0,1949,4,0,3
4,1949-05-01,121.0,1949,5,0,4


### Split train/test sets

In [3]:
max_encoder_length = 14
max_prediction_length = 12

training_cutoff = data["time_idx"].max() - max_encoder_length - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    max_encoder_length= max_encoder_length,
    min_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    min_prediction_length=max_prediction_length,
    time_idx="time_idx",
    target="Passengers",
    group_ids=["group"],
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=['month'],
    time_varying_known_reals=[],
    time_varying_unknown_reals=["Passengers"],
    time_varying_unknown_categoricals=[],
    target_normalizer=TorchNormalizer(method="standard",
                                      transformation=None),
    )

training.get_parameters()
validation = TimeSeriesDataSet.from_dataset(training,
                                            data[lambda x: x.time_idx > training_cutoff])

batch_size = 16
train_dataloader = DataLoader(training, batch_size=batch_size, shuffle=False, drop_last=False)
val_dataloader = DataLoader(validation, batch_size=batch_size, shuffle=False, drop_last=False)


### Define model

In [4]:
pl.seed_everything(123)
# create PyTorch Lighning Trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4,
                                    patience=60, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

trainer = pl.Trainer(
    max_epochs=300,
    gpus=0,  # run on CPU, if on multiple GPUs, use accelerator="ddp"
    gradient_clip_val=0.1,
    limit_train_batches=30,  # 30 batches per epoch
    callbacks=[lr_logger, early_stop_callback],
    logger=TensorBoardLogger("lightning_logs")
)

model = Seq2SeqNetwork.from_dataset(training,
                                    hidden_size=50,
                                    rnn_layers=3)
model.summarize

Global seed set to 123
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


<bound method LightningModule.summarize of Seq2SeqNetwork(
  (loss): L1Loss()
  (logging_metrics): ModuleList()
  (dense_layer): Linear(in_features=50, out_features=1, bias=True)
  (encode_rnn): LSTM(7, 50, num_layers=3, batch_first=True)
  (decode_rnn): LSTM(6, 50, num_layers=3, batch_first=True)
  (embeddings): ModuleDict(
    (month): Embedding(12, 6)
  )
)>

In [5]:
model.hparams

"categorical_groups":                {}
"cell_type":                         LSTM
"embedding_labels":                  {'month': array(['1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype=object)}
"embedding_paddings":                []
"embedding_sizes":                   {'month': [12, 6]}
"hidden_size":                       50
"learning_rate":                     0.001
"log_interval":                      -1
"log_val_interval":                  None
"logging_metrics":                   ModuleList()
"loss":                              L1Loss()
"max_encoder_length":                14
"max_prediction_length":             12
"monotone_constaints":               {}
"output_size":                       1
"output_transformer":                TorchNormalizer()
"rnn_layers":                        3
"static_categoricals":               []
"static_reals":                      []
"time_varying_categoricals_decoder": ['month']
"time_varying_categoricals_encoder": ['mon

### Train model with early stopping

In [6]:
trainer.fit(
    model, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
)

# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = Seq2SeqNetwork.load_from_checkpoint(best_model_path)

# calcualte mean absolute error on validation set
actuals = torch.cat([model.transform_output(prediction=y, target_scale=x['target_scale'])
                     for x, y in iter(val_dataloader)])
predictions, x_index = best_model.predict(val_dataloader)
mae = (actuals - predictions).abs().mean()
# print('predictions shape is:', predictions.shape)
# print('actuals shape is:', actuals.shape)
print(torch.cat([actuals, predictions]))
print('MAE is:', mae)

D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(

  | Name            | Type       | Params
-----------------------------------------------
0 | loss            | L1Loss     | 0     
1 | logging_metrics | ModuleList | 0     
2 | dense_layer     | Linear     | 51    
3 | encode_rnn      | LSTM       | 52.6 K
4 | decode_rnn      | LSTM       | 52.4 K
5 | embeddings      | ModuleDict | 72    
-----------------------------------------------
105 K     Trainable params
0         Non-trainable params
105 K     Total params
0.420     Total estimated model params size (MB)


D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 123
D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:432: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_st

Epoch 0:  86%|█████████▍ | 6/7 [00:00<00:00, 26.90it/s, loss=0.742, v_num=13]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|█| 7/7 [00:00<00:00, 29.66it/s, loss=0.742, v_num=13, val_loss=
Epoch 1:  86%|▊| 6/7 [00:00<00:00, 20.00it/s, loss=0.734, v_num=13, val_loss=
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|█| 7/7 [00:00<00:00, 22.08it/s, loss=0.734, v_num=13, val_loss=
Epoch 2:  86%|▊| 6/7 [00:00<00:00, 11.89it/s, loss=0.731, v_num=13, val_loss=
Validating: 0it [00:00, ?it/s]
Epoch 2: 100%|█| 7/7 [00:00<00:00, 13.22it/s, loss=0.731, v_num=13, val_loss=
Epoch 3:  86%|▊| 6/7 [00:00<00:00, 13.23it/s, loss=0.77, v_num=13, val_loss=2
Validating: 0it [00:00, ?it/s]
Epoch 3: 100%|█| 7/7 [00:00<00:00, 14.69it/s, loss=0.77, v_num=13, val_loss=2
Epoch 4:  86%|▊| 6/7 [00:00<00:00, 12.19it/s, loss=0.768, v_num=13, val_loss=
Validating: 0it [00:00, ?it/s]
Epoch 4: 100%|█| 7/7 [00:00<00:00, 13.64it/s, loss=0.768, v_num=13, val_loss=
Epoch 5:  86%|▊| 6/7 [00:00<00:00, 13.59it/s, loss=0.765, v_num=1